**Sorry if this notebook looks kinda messy and also sorry for my bad english**
<br>~
<br>Sources/Credits :
<br>FrameBot by [Boidushya](https://github.com/Boidushya)
<br>Colab file made by [izu14](https://github.com/izu14)
<br>~
<br>Before proceeding, please read this [README.md](https://github.com/izu14/FrameBot-Colab/blob/master/README.md)

**!!!RUN THE COMMAND ACCORDING TO ORDER NUMBER (1 > 2 > 3)!!!**

**[1] CLONE FRAMEBOT REPO AND INSTALL REQUIRED PACKAGES**

In [ ]:
#@markdown ### ← Run This
from IPython.display import clear_output
!git clone https://github.com/izu14/FrameBot
clear_output()
%mkdir /content/FrameBot/assets/video
%cd /content/FrameBot
clear_output()
!sudo pip3 install -r requirements.txt
clear_output()
!sudo pip3 install -q youtube-dl
clear_output()
print(f"Done")

**[2] CHOOSE VIDEO SOURCE(Only Support Single Video Input)**

In [ ]:
# @markdown ###← YouTube or any [ytdl supported sites](https://gitlab.com/ytdl-org/youtube-dl/-/raw/master/docs/supportedsites.md)
from IPython.display import clear_output
%rm /content/FrameBot/assets/video/*.mp4 
Link="putvideolinkhere"#@param{type:"string"}
!youtube-dl -f 'bestvideo[ext=mp4]' --output "/content/FrameBot/assets/video/%(title)s.%(ext)s" $Link
clear_output()
!for filename in /content/FrameBot/assets/video/*.mp4; do echo 'Downloaded Video to '"$filename"; done


In [ ]:
#@markdown ###← Local File
from IPython.display import clear_output
import google.colab.files
%rm /content/FrameBot/assets/video/*.mp4
%cd /content/FrameBot/assets/video
clear_output()
uploaded = google.colab.files.upload()

**[3] FILL ALL REQUIRED VALUE THEN PRESS RUN/PLAY BUTTON TO START THE BOT**

In [ ]:
#This code was written by Boidushya Bhattacharya and Gustav Wallström (github/sudoxd) on Monday, 26 November 2019 at 20:27 p.m.
#Reddit: https://reddit.com/u/Boidushya
#Facebook: https://facebook.com/soumyadipta.despacito
 
#@markdown ###← Press this after you fill all the required Value
from IPython.display import clear_output
import cv2
import os
import math
import facebook
import functools
import schedule
import time
import fnmatch
import sys
 
#@markdown ###<br> ↓ Replace value below with your facebook token
Token = "putyourtokenhere" #@param{type:"string"}
%store Token > /content/FrameBot/assets/token.txt
clear_output()
#@markdown > How to get your own token : <br>https://github.com/Boidushya/FrameBot/blob/master/generateToken.md</p>
 
#@markdown ###<br> ↓ Configure your post caption
Title = "aiueo"#@param{type:"string"}
#@markdown > Default input(aiueo) will give a result like : <br>aiueo - Frame 1 out of 2
 
#@markdown ###<br> ↓ Change the FPS for your video here
FPS=1#@param{type:"number"}
#@markdown > The more the fps, the more number of frames, the longer it'll take to post the whole frame

#@markdown ###<br> Interval/delay(in Seconds) for each post
Interval=15#@param{type:"number"}
 
def catch_exceptions(cancel_on_failure=False):
    def catch_exceptions_decorator(job_func):
        @functools.wraps(job_func)
        def wrapper(*args, **kwargs):
            try:
                return job_func(*args, **kwargs)
            except:
                import traceback
                print(traceback.format_exc())
                if cancel_on_failure:
                    return schedule.CancelJob
        return wrapper
    return catch_exceptions_decorator
 
def extractFrames():
    file = os.listdir('/content/FrameBot/assets/video')[0]
    videoFile = f"/content/FrameBot/assets/video/{file}"
    if not os.path.exists('/content/FrameBot/assets/frames'):
        os.mkdir('/content/FrameBot/assets/frames')
    if os.path.exists("/content/FrameBot/assets/frames/*.jpg"):
        os.remove("/content/FrameBot/assets/frames/*.jpg")
    vidcap = cv2.VideoCapture(videoFile)
    success,image = vidcap.read()
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    required_fps = FPS
    multiplier = round(fps/required_fps)
    x=0
 
    while success:
        frameId = int(round(vidcap.get(1)))
        success, image = vidcap.read()
 
        if frameId % multiplier == 0:
            x+=1
            cv2.imwrite(f"/content/FrameBot/assets/frames/frame{int(x):06d}.jpg", image)
    vidcap.release()
 
@catch_exceptions()
def post():
    dir = os.listdir("/content/FrameBot/assets/frames")
    dir.sort(key = lambda t : int(t[5:-4])) #forgot to sort the files before, pls forgive me lol
    with open("/content/FrameBot/assets/retain","a+") as f:
        f.seek(0)
        filled = f.read(1)
        if not filled:
            totalFrames = str(len(dir))
            f.write(totalFrames)
        else:
            f.seek(0)
            totalFrames = str(f.readline())
 
    currentFrame = f'/content/FrameBot/assets/frames/{dir[0]}'
    currentFrameNumber = str(int(dir[0][5:-4]))
    msg = f"{Title} - Frame {currentFrameNumber} out of {str(totalFrames)}"
    with open('/content/FrameBot/assets/token.txt','r') as token:
        accesstoken = token.readline()
    graph = facebook.GraphAPI(accesstoken)
    post_id = graph.put_photo(image=open(currentFrame, 'rb'),message = msg)['post_id']
    print(f"Submitted post with title \"{msg}\" successfully!")
    os.remove(currentFrame)
 
if __name__ == '__main__':
    token = open('/content/FrameBot/assets/token.txt', 'r')
    if token.readline() == "putyourtokenhere":
        print("please replace putyourtokenhere with your own token")
        sys.exit("error no token")
    ans = "y"
    if 'y' in ans.lower():
        if os.path.exists("/content/FrameBot/assets/retain"):
            os.remove("/content/FrameBot/assets/retain")
        extractFrames()
    else:
        pass
    schedule.every(Interval).seconds.do(post).run()
    #schedule.every(Interval).seconds.do(post).run()
 
    while 1:
        schedule.run_pending()
        time.sleep(1)

#### <br>**AFTER THE LAST FRAME POSTED,**
#### **YOU MUST PRESS THE STOP BUTTON(IT WAS RUN/PLAY BUTTON PREVIOUSLY) ON THE LEFT TO STOP THIS CODE**
#### **OR RIGHT CLICK ON IT THEN CLICK INTERRUPT EXECUTION**
#### **ELSE, IT'LL ALWAYS THROW/SHOW AN ERROR**